## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제 없음

In [20]:
from urllib.request import urlopen # python 3

# 도시별 날씨 검색 함수 13번 돌 수 있음
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    #print(page.read().decode('utf-8')) 
    text = (page.read().decode('utf-8')) 
    #print(text[text.find(f'>{city}</a>'):])
    text = text[text.find(f'>{city}</a>'):]

    # 기온 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    # print(text[start:end])
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

    # 습도 가져오기
    for i in range(3):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    # print(text[start:end])
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')

get_weather('부산')


부산의 현재 기온은 7.1˚C 입니다.
부산의 현재 습도는 41% 입니다.


### OpenAPI 크롤링

- 공공데이터 포털
    - http://www.data.go.kr
    - 부산광역시 버스정보안내기 현황


```python
response = requests.get(total_url, verify = False)
```
- 문제점
    - http를 일반 request로 부르면 SSL 오류발생
    - 해결하려면 외부모듈 requests를 사용, verify=False 옵션을 지정
    - 아래 셀의 19번 라인!
    

In [73]:
import requests  # Request => response 사용해서 더 정확하고 안전하게 사용할 수 있음
from urllib.parse import quote, unquote, urlencode # quote => 한글을 url 인코드로 변환하는 함수 부경대 --> %EB%B6%80%EA%B2%BD%EB%8C%80
# urllib 웹과 관련된 데이터를 쉽게 다룰 수 있도록 하는 라이브러리 url 파싱과 관련된 것 => parse
import json

    # stationoName : 정류장 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(
        {
        'serviceKey' : 'LRZBDK0Pybz4yk9DMkGqYmde2FvwRLT82e5sRo2DByH89ICaW6A/bLNl2Ehe7O/dj78UMA06t2eJ0Rw5k1GwHQ==',#url 인코드를 써서 디코딘된 ㄱㅄ을 써야함
        'pageNo' : '1',
        'numOfRows' : '10',
        'resultType' : type,
        'stationLoc' : stationName
        }
    )
    
    total_url = api_url + queryString
    # print(total_url)
    response = requests.get(total_url, verify = False)
    return(response.text)

try:
    result = getDataPortalSearch('부경대', 'json')
    json_data = json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    for i in station_data:
        print(i)
except Exception as e:
    print('찾는 데이터가 없습니다.')


{'stationNum': '07006', 'stationLoc': '경성대.부경대역', 'lat': '35.13856124', 'lng': '129.1023074', 'addr': '남구 대연3동 93-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07007', 'stationLoc': '경성대.부경대역', 'lat': '35.13837971', 'lng': '129.1024864', 'addr': '남구 대연3동 90-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07011', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13287689', 'lng': '129.1012804', 'addr': '남구 대연3동 531-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07012', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13234515', 'lng': '129.1010586', 'addr': '남구 대연3동 537-1', 'insYear': '', 'dataDay': '2023-02-03'}


### OpenAPI로 가져온 데이터를 지도에 표시하기

```python
stop_str = '<h4>' + item['stationLoc'] +'</h4>' + item['stationNum']+'<br>'+item['addr']
```

- 문제점
    - 딕셔너리 구조하고 문자열 포캣팅 f'{}' 호환 안됨!
    - 구식방법인 무자열 결합방식으로 해결


In [74]:
import folium

if len(station_data) > 0: # 정류소 중 제일 첫번째 인덱스에 있는 정류소 위경도를 중심으로 잡은 것
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']

m = folium.Map(location = [center_lat, center_lng], zoom_start=13)

# 전체 정류소 위치값 마커 표시
for item in station_data:
    stop_str = '<h4>' + item['stationLoc'] +'</h4>' + item['stationNum']+'<br>'+item['addr']
    iframe = folium.IFrame (stop_str)
    popup = folium.Popup(iframe, min_width=200, max_width=200)
    folium.Marker(location=[item['lat'],item['lng']], popup=popup,
                  icon = folium.Icon(icon='pushpin')).add_to(m)
m

### BeautifulSoup (version 4)

웹크롤링을 편하게 해주는 도구

```shell
pip install beautifulsoup4
```


In [2]:
!pip install beautifulsoup4


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 모듈 import

```python
from bs4 import BeautifulSoup
```


In [6]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)
if response.status_code ==200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') # pasrser : 어떤 값들을 분석해서 우리가 보기 좋도록 구조화 시켜줌
else:
    print(f'Error : {response.status_code}')

soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta contents="always" name="referrer"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="제임스 웹 망원경의 지식iN Q&amp;A 검색결과입니다. 궁금증을 해결하지 못했다면 지식iN '질문하기'를 해보세요." name="description">
<meta content="width=1024" name="viewport"/>
<meta content="none" name="msapplication-config">
<link href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/components.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/other.css" rel="stylesheet" type="text/css"/>
<script>
	
	var pcDomain = "kin.naver.com";
	var pcDomainWithProtocol = "https://kin.naver.com";
	var pcKinServiceProtocol = "https";

	var mobileDomain = "m.kin.naver.com";
	var mobileDomainWithProtocol = "https://m.kin.naver.com";
	var mobileKinServiceProtocol = "https";

	var